In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     |████████████████████████████████| 1904.6 MB 37 kB/s s eta 0:00:01    |█████▎                          | 315.9 MB 19.5 MB/s eta 0:01:22     |███████▏                        | 424.3 MB 11.4 MB/s eta 0:02:11     |██████████▉                     | 643.0 MB 11.3 MB/s eta 0:01:52     |████████████████▍               | 977.8 MB 13.9 MB/s eta 0:01:07     |███████████████████             | 1132.5 MB 13.8 MB/s eta 0:00:56     |████████████████████            | 1195.9 MB 7.3 MB/s eta 0:01:38     |█████████████████████▏          | 1260.6 MB 18.3 MB/s eta 0:00:36     |█████████████████████▌          | 1280.2 MB 11.3 MB/s eta 0:00:56     |███████████████████████▍        | 1392.5 MB 13.9 MB/s eta 0:00:37     |█████████████████████████▊      | 1527.7 MB 14.4 MB/s eta 0:00:27     |███████████████████████████████▌| 1876.5 MB 8.9 MB/s eta 0:00:04
     |████████████████████████████████| 23.5 MB 14.8 MB/s eta 0:00:01


In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.0107, 0.5797, 0.7397],
        [0.6336, 0.6756, 0.4482],
        [0.5044, 0.0788, 0.7751],
        [0.5465, 0.6859, 0.8449],
        [0.6687, 0.0436, 0.2197]])


In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291355  [    0/60000]
loss: 2.277359  [ 6400/60000]
loss: 2.252076  [12800/60000]
loss: 2.251232  [19200/60000]
loss: 2.226559  [25600/60000]
loss: 2.191054  [32000/60000]
loss: 2.210871  [38400/60000]
loss: 2.160328  [44800/60000]
loss: 2.161102  [51200/60000]
loss: 2.133688  [57600/60000]
Test Error: 
 Accuracy: 42.4%, Avg loss: 2.119886 

Epoch 2
-------------------------------
loss: 2.128330  [    0/60000]
loss: 2.114535  [ 6400/60000]
loss: 2.046214  [12800/60000]
loss: 2.076935  [19200/60000]
loss: 2.002991  [25600/60000]
loss: 1.938003  [32000/60000]
loss: 1.983928  [38400/60000]
loss: 1.883018  [44800/60000]
loss: 1.900062  [51200/60000]
loss: 1.831867  [57600/60000]
Test Error: 
 Accuracy: 56.2%, Avg loss: 1.820826 

Epoch 3
-------------------------------
loss: 1.855044  [    0/60000]
loss: 1.818329  [ 6400/60000]
loss: 1.694518  [12800/60000]
loss: 1.753472  [19200/60000]
loss: 1.621786  [25600/60000]
loss: 1.590352  [32000/600

In [14]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [15]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [16]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [19]:
import torch
import numpy as np

data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

np_array = np.array(data)
x_np = torch.from_numpy(np_array)

x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.2968, 0.0777],
        [0.5536, 0.9179]]) 



In [22]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")
print(f"{tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu
cuda:0
